Ventas de Video Juegos

Tabla obtenida de Kaggle
Dataset contiene un listado de videojuegos con ventas mayores a 100,000 copias. Generado de un webscrap de vgchartz.com
https://www.kaggle.com/datasets/gregorut/videogamesales

Datos actualizados al 2016.

Analisis desde el punto de vista de empleado de PlayStation y ver donde se deben enfocar los próximos videojuegos y en que géneros

In [ ]:
#Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests
import seaborn as sns

In [ ]:
#Assign csv file containing vg dataset to the variable df (previously uploaded to personal Google Drive) and check the information format
df = pd.read_csv('G:/My Drive/Proyecto/vgsales.csv')
df.head()

In [ ]:
#Check all the DataFrame information, shape: 11 columns, 16,598 rows
df.info()

In [ ]:
#As an analyst for Playstation in this project I will be analyzing only Sony Playstation Sales but dropping PS1 as it was released over 20 years ago
ps_df = df[(df['Platform'] == 'PS4') | (df['Platform'] == 'PS3') | (df['Platform'] == 'PS2')]
ps_df.sort_values('Global_Sales', ascending= False).head()

In [ ]:
#Initial exploratory analysis. 5022 rows (some missing values). Year has a float format. 
ps_df.info()
ps_df.describe()

In [ ]:
#Double check visually
plt.figure(figsize= (10, 6))
sns.regplot(data= ps_df, y= "Global_Sales", x= "Year", fit_reg= False)
plt.show()

In [ ]:
#Function to search an API on https://rawg.io/ to find the missing release year of the games.

# Replace 'RAWG_API_KEY'
RAWG_API_KEY = '284f997010e446a6a4782c2f1e11f196'

# List of game names
games = [
    'Madden NFL 2004', 'FIFA Soccer 2004', 'wwe Smackdown vs. Raw 2006', 'Rock Band',
    'Triple Play 99', 'NASCAR Thunder 2003', 'Rock Band', 'Legacy of Kain: Soul Reaver',
    'Suikoden III', 'Yakuza 4', 'LEGO Harry Potter: Years 5-7', 'Wheel of Fortune',
    'The Lord of the Rings: War in the North', 'MLB SlugFest 20-03', 'Test Drive Unlimited 2',
    'Monster Hunter 2', 'NASCAR: Dirt to Daytona', 'Sonic the Hedgehog', 'NFL GameDay 2003',
    'Silent Hill: Homecoming', 'Harvest Moon: Save the Homeland', 'Robert Ludlum\'s The Bourne Conspiracy',
    'Final Fantasy XI', 'All-Star Baseball 2005', 'The Dukes of Hazzard II: Daisy Dukes It Out',
    'Haven: Call of the King', 'Rock Revolution', 'Jonah Lomu Rugby Challenge', 'College Hoops 2K6',
    'Mega Man X Collection', 'Singularity', 'Jet X20', 'Tribes: Aerial Assault', 'Move Fitness',
    'Yu Yu Hakusho: Dark Tournament', 'WRC: FIA World Rally Championship', 'Twisted Metal: Small Brawl',
    'NBA Starting Five', 'Backbreaker', 'James Cameron\'s Dark Angel', 'Sword of the Samurai',
    'Splatterhouse', 'Alone in the Dark: The New Nightmare', 'eJay Clubworld', 'Bejeweled 3',
    'Record of Agarest War Zero', 'Saru! Get You! Million Monkeys', 'Star Trek: Conquest', 'Happy Feet Two',
    'Demon Chaos', 'Action Man-Operation Extreme', 'McFarlane\'s Evil Prophecy', 'Port Royale 3',
    'Combat Elite: WWII Paratroopers', 'Mountain Bike Adrenaline', 'Battle vs. Chess',
    'The History Channel: Great Battles - Medieval', 'Monster Hunter Frontier Online', 'B.L.U.E.: Legend of Water',
    'Sega Rally 2006', 'Samurai Spirits: Tenkaichi Kenkakuden', 'Umineko no Naku Koro ni San: Shinjitsu to Gensou no Yasoukyoku',
    'Cabela\'s Alaskan Adventure', 'Virtua Quest', 'Agarest Senki: Re-appearance', 'Hakuouki: Shinsengumi Kitan'
]

def get_game_release_year(game_name):
    url = f'https://api.rawg.io/api/games?search={game_name}&key={RAWG_API_KEY}'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if 'results' in data and data['results']:
            release_year = data['results'][0].get('released')
            if release_year:
                return release_year[:4]
            else:
                print(f"Release year not found for {game_name}.")
        else:
            print(f"Data not found for {game_name}.")
    else:
        print(f"Error fetching data for {game_name}. Status code: {response.status_code}")
        print("Response text:", response.text)

    return None

# Create an empty dictionary to store the game names and release years
games_release_years = {}

# Get release years for each game and save them in the dictionary
for game in games:
    release_year = get_game_release_year(game)
    games_release_years[game] = release_year

In [ ]:
#Two games were not found on the API: McFarlane's Evil Prophecy (2004) and Combat Elite: WWII Paratroopers (2005), these were changed manually

#games_release_years['McFarlane's Evil Prophecy']
                    
games_release_years.update({"McFarlane\'s Evil Prophecy": "2004","Combat Elite: WWII Paratroopers": "2005",})
print(games_release_years['McFarlane\'s Evil Prophecy'])
print(games_release_years['Combat Elite: WWII Paratroopers'])

In [ ]:
#Fill empty values of year with the dictionary games_release_years previously filled and replace Publisher values with "Unknown"
#Found game with the wrong Genre classification FIFA Soccer 13 --> Action  --> Sports 
ps_df['Year'] = ps_df['Year'].fillna(ps_df['Name'].map(games_release_years))
ps_df['Publisher'] = ps_df['Publisher'].fillna('Unknown')
ps_df.isnull().sum()

ps_df.loc[ps_df["Name"] == "FIFA Soccer 13", "Genre"] = "Sports"

In [ ]:
#Check again all DataFrame's info
ps_df.info()

In [ ]:
ps_df = ps_df.astype({'Year': 'int'})
ps_df['Year']

In [ ]:
#Check again all DataFrame's info, changed Year to integer
ps_df.info()

In [ ]:
#Start exploratory graphs

sns.histplot(x=ps_df['Platform'].sort_values())

plt.title("Releases by platform")
plt.xlabel("PlayStation Platform")
plt.ylabel("Number of games")

plt.show()

In [ ]:
plt.figure(figsize= (10,6))
sns.histplot(x=ps_df['Genre'])


plt.title("Releases by Genre")
plt.xlabel("Game Genre")
plt.ylabel("Number of games")
plt.xticks(rotation=45)

plt.show()

In [ ]:
#Group the DataFrame by genre to see which genre is leading sales
sales_per_genre_ww = ps_df.groupby('Genre')['Global_Sales'].sum().reset_index()

custom_palette = sns.color_palette('Set3')

plt.figure(figsize= (10, 8))
plt.pie(sales_per_genre_ww['Global_Sales'], labels= sales_per_genre_ww['Genre'], autopct= '%1.1f%%', colors= custom_palette)

plt.title("Sales by Genre Worldwide")
plt.show()

In [ ]:
#Group the DataFrame by genre to see which genre is leading sales
sales_per_genre_na = ps_df.groupby('Genre')['NA_Sales'].sum().reset_index()

plt.figure(figsize= (10,6))
plt.bar(sales_per_genre_na['Genre'], sales_per_genre_na['NA_Sales'])

plt.title("Sales by Genre in North America")
plt.xlabel("Game Genre")
plt.ylabel("Number of copies sold (in Millions)")
plt.xticks(rotation=45)

plt.show()

In [ ]:
#Group the DataFrame by genre to see which genre is leading sales
sales_per_genre_eu = ps_df.groupby('Genre')['EU_Sales'].sum().reset_index()

plt.figure(figsize= (10,6))
plt.bar(sales_per_genre_eu['Genre'], sales_per_genre_eu['EU_Sales'])

plt.title("Sales by Genre in Europe")
plt.xlabel("Game Genre")
plt.ylabel("Number of copies sold (in Millions)")
plt.xticks(rotation=45)

plt.show()

In [ ]:
#Group the DataFrame by genre to see which genre is leading sales
sales_per_genre_jp = ps_df.groupby('Genre')['JP_Sales'].sum().reset_index()

plt.figure(figsize= (10,6))
plt.bar(sales_per_genre_jp['Genre'], sales_per_genre_jp['JP_Sales'])

plt.title("Sales by Genre in Japan")
plt.xlabel("Game Genre")
plt.ylabel("Number of copies sold (in Millions)")
plt.xticks(rotation=45)

plt.show()

In [ ]:
#Group the DataFrame by genre to see which genre is leading sales
sales_per_genre_other = ps_df.groupby('Genre')['Other_Sales'].sum().reset_index()

plt.figure(figsize= (10,6))
plt.bar(sales_per_genre_other['Genre'], sales_per_genre_other['Other_Sales'])

plt.title("Sales by Genre in the rest of the World")
plt.xlabel("Game Genre")
plt.ylabel("Number of copies sold (in Millions)")
plt.xticks(rotation=45)

plt.show()

In [ ]:
#Survey the best selling games on the top categories
ps_df[ps_df['Genre'] == "Action"].sort_values('Global_Sales', ascending= False).head(10)

In [ ]:
ps_df[ps_df['Genre'] == "Sports"].sort_values('Global_Sales', ascending= False).head()

In [ ]:
ps_df[ps_df['Genre'] == "Shooter"].sort_values('Global_Sales', ascending= False).head()

In [ ]:
ps_df[ps_df['Genre'] == "Racing"].sort_values('Global_Sales', ascending= False).head()

In [ ]:
ps_df[ps_df['Genre'] == "Role-Playing"].sort_values('Global_Sales', ascending= False).head()

In [ ]:
sales_per_publisher = ps_df.groupby("Publisher")["Global_Sales"].sum().reset_index()

sales_per_publisher.sort_values("Global_Sales", ascending= False).head(10)

In [ ]:
ps_df[ps_df["Publisher"] == "Sony Computer Entertainment"].sort_values("Global_Sales", ascending= False).head(10)

In [ ]:
ps_df[ps_df["Name"].str.contains("Uncharted")].sort_values("Global_Sales", ascending= False).head(10)

In [ ]:
sales_per_year = ps_df[ps_df["Genre"] == "Racing"].groupby("Year").agg({
    "Genre": "first",
    "Global_Sales": "sum"
}).reset_index()

sales_per_year

In [ ]:
sales_gran_turismo = ps_df[ps_df["Name"].str.contains("Gran Turismo")]

sales_gran_turismo[["Name", "Year", "Global_Sales"]]

In [ ]:
plt.figure(figsize= (20, 6))
sns.lineplot(data= sales_per_year, x= "Year", y="Global_Sales")
bar_width = 0.4
plt.bar(sales_gran_turismo["Year"], sales_gran_turismo["Global_Sales"], color= "red", alpha= 0.7, width= 0.4)

plt.xlabel("Year")
plt.ylabel("Sales")
plt.title("Racing Sales per Year v Gran Turismo releases")
plt.xticks(sales_per_year["Year"])

plt.show()


Conclusiones y recomendaciones:

Las cinco categorías mas fuertes en ventas a nivel mundial son:
    
    -Acción
    -Deportes
    -Shooters (Tiradores)
    -Carreras
    -RPG (Juegos de Rol)


Todos los géneros tienen una franquicia que sobresale en ventas de las demás:
    
    -Acción         --> Grand Theft Auto
    -Deportes       --> Fifa
    -Shooters       --> Call of Duty
    -Carreras       --> Gran Turismo
    -RPG            --> Final Fantasy 

Nos encontramos en segundo lugar en ventas como casa fabricante:

    -Electronic Arts
    -Sony Computer Entertainment
    -Activision




Como empresa, Sony Computer Entertainment debemos enfocarnos en las siguientes franquicias:
    
    - Gran Turismo, el último juego se estrenó en 2010, sin  embargo la franquicia sigue en auge ya que se estrenará una película en un par de semanas. Hay que considerar que lidera el rubro de la categoría de carreras y a pesar de que ha venido en caída, siempre que se libera un juego nuevo de Gran Turismo las ventas aumentan considerablemente.
    - Uncharted, el último juego fue liberado en 2016 y se ha vuelto muy popular debido a la película del año 2022. Se podría experimentar con un model open-world siguiendo los linbeamientos de Grand Theft Auto que es la franquicia más popular en juegos de acción.